In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint    
import seaborn as sns

%matplotlib inline

# Pull Data

In [2]:
path = "C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/"
test_df = pd.read_csv(path+'test.csv')
train_df = pd.read_csv(path+'train.csv')
validation_df = pd.read_csv(path+'validation.csv')

In [3]:
TotalImpressions = test_df.shape[0]
print("TotalImpressions:",TotalImpressions) #create a function for this


303375

In [4]:
budget=6250*1000
print("Our Budget is:",budget, "Fen")
print("To use over",test_df.shape[0],"bid opportunities.")
BudPerBid=budget/test_df.shape[0]
print("Budget / Bid Opportunity:",BudPerBid)

minBid=np.min(train_df["payprice"].values)
maxBid=np.max(train_df["payprice"].values)
custRange = np.arange(minBid+1, maxBid+2,10) # determines the range that bids should be in

Our Budget is: 6250000 Fen
To use over 303375 bid opportunities.
Budget / Bid Opportunity: 20.601565718994642


NameError: name 'validation_df' is not defined

In [ ]:
pd.set_option('display.max_columns', None)

# Functions

### Plot Metrics for bound combination for random bidding strategy

In [ ]:
def plotResults(Matrix):

    ax = sns.heatmap(Matrix, linewidth=0, xticklabels=custRange, yticklabels=custRange[::-1], cmap="Greens") #, annot=True
    ax.set(xlabel='lower bound for random bid', ylabel='upper bound for random bid', title="Clicks by bounded random bids")
    for label in ax.xaxis.get_ticklabels()[::1]:
        label.set_visible(False)
    for label in ax.xaxis.get_ticklabels()[::1]:
        label.set_visible(True)
    for label in ax.yaxis.get_ticklabels()[::1]:
        label.set_visible(False)
    for label in ax.yaxis.get_ticklabels()[::1]:
        label.set_visible(True)
    plt.show()
    
    useless = 0
    return useless

### Build Matrix containing CTR for various bound combinations

In [ ]:
def BuildCTRMatrix(dataframe,plot,repeats):
    for n in range(0,repeats,1):
        dataSubset = dataframe.sample(frac=0.5, replace=False)
        print(n+1,"/",repeats,"...")
        for minimumBid in custRange:
            for maximumBid in custRange: #maxBid>minBid?????
                if maximumBid>minimumBid:
                    ClickThroughRate = EvalRandBid(dataSubset,minimumBid, maximumBid)
                    CTRMatrix[minimumBid][maximumBid]=(CTRMatrix[minimumBid][maximumBid]*n+ClickThroughRate)/(n+1)
   
    return CTRMatrix

### Extract best Bounds from Results Matrix

In [ ]:
def FindBestBounds(ResultsMatrix):
    print("Best CTR is:",ResultsMatrix.values.max())
    for x in custRange:
        for y in custRange:
            if ResultsMatrix[x][y]==ResultsMatrix.values.max():
                print("X:",x,"Y:",y)
                lowerBound=x
                upperBound=y
    return lowerBound, upperBound

### Evaluate a random bid value on a new set

In [ ]:
def EvalRandBid(dataframe,lowerBound, upperBound):
    
    dataframe['ConstBid'] = np.random.randint(lowerBound, upperBound+1, dataframe.shape[0])

    trueValues = (dataframe["payprice"]<dataframe['ConstBid'])
    impressions = dataframe[trueValues].shape[0]
    clicks = np.sum(dataframe[trueValues]["click"].values)
    ClickThroughRate=(clicks*100)/impressions
        
    return ClickThroughRate

# Main Code

### Determine Best Bounds Combination from Train Set

In [ ]:
zero_data = np.zeros(shape=(len(custRange),len(custRange)))
zero_data[zero_data < 1] = 0.05
CTRMatrix=pd.DataFrame(zero_data,index=reversed(custRange), columns=custRange)

In [ ]:
CTRMatrix =  BuildCTRMatrix(train_df,plot=False,repeats=15)
#CTRMatrix.to_pickle("CTRMatrix.pkl")

In [ ]:
#CTRMatrix = pd.read_pickle("CTRMatrix.pkl")

In [ ]:
plotResults(CTRMatrix)

In [ ]:
lowBound, upBound = FindBestBounds(CTRMatrix)  

### Evalute on a new data set (validation set)

In [ ]:
CTR = EvalRandBid(validation_df,lowBound, upBound)

In [ ]:
print(CTR)